In [0]:
from __future__ import absolute_import, division, print_function

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
print(tf.__version__)

In [0]:
# Recoleccion de la data
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [0]:
# Normalizacion de datos
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train.shape, x_test.shape

In [0]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [0]:
# Data ingest
def read_dataset(x, y, mode, batch_size=20):
  x_ds = tf.data.Dataset.from_tensor_slices(x)
  y_ds = tf.data.Dataset.from_tensor_slices(tf.cast(y, tf.int64))
  
  ds = tf.data.Dataset.zip((x_ds, y_ds))
  
  if mode == tf.estimator.ModeKeys.TRAIN:
    num_epochs = None
    ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=batch_size*10))
  else:
    num_epochs = 1
  
  ds = ds.batch(batch_size).repeat(num_epochs).prefetch(tf.data.experimental.AUTOTUNE)
  return ds

def train_input_fn():
  return read_dataset(x = x_train,#<-------
                      y = y_train,#<-------
                      mode = tf.estimator.ModeKeys.TRAIN)

def eval_input_fn():
  return read_dataset(x = x_test,#<-------
                      y = y_test,#<-------
                      mode = tf.estimator.ModeKeys.EVAL)

In [0]:
# Modelo: Keras creacion
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

estimator = tf.keras.estimator.model_to_estimator(keras_model=model)

In [0]:
# Produccion: data
train_spec = tf.estimator.TrainSpec(input_fn = train_input_fn,
                                   max_steps = 20)
eval_spec = tf.estimator.EvalSpec(input_fn = eval_input_fn)

In [0]:
# Entrenamiento
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)